Automobile Sales Forecasting Using Functional Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skfda
from skfda.preprocessing.dim_reduction.feature_extraction import FPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import adam_v2
from keras.layers import LSTM
from math import sqrt
from scipy.stats import gaussian_kde
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from skfda.ml.regression import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from skfda.misc.metrics import l2_distance, l2_norm
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [1]:
#Table 2: FDA models using different durations of sales data

In [2]:
data_matrix_train = pd.read_csv('data_matrix_train.csv')
del data_matrix_train['Unnamed: 0']
data_matrix_test = pd.read_csv('data_matrix_test.csv')
del data_matrix_test['Unnamed: 0']
sales = pd.concat([data_matrix_train,data_matrix_test], axis=1)
sales.columns = range(1,41)
grid_points = []
for i in range(1,41):
    grid_points.append(i)
fd_y = skfda.FDataGrid(
    data_matrix=sales,
    grid_points=grid_points,
)
q = 4
fpca_clean = FPCA(n_components=q)
fpca_clean.fit(fd_y)
fd_y_hat = fpca_clean.inverse_transform(
    fpca_clean.transform(fd_y)
)
err_fd_y = l2_distance(
    fd_y,
    fd_y_hat
) / l2_norm(fd_y)
x_density = np.linspace(0., 1.6, num=2276)
density_err_fd_y = gaussian_kde(err_fd_y)
err_thresh = np.quantile(err_fd_y, 0.95)
newsales = sales[err_fd_y < err_thresh]

In [13]:
#FDA 12months

In [14]:
fd_y = skfda.FDataGrid(
    data_matrix=newsales.iloc[:,36:40],
    grid_points=grid_points[36:40],
)
fd_x = skfda.FDataGrid(
    data_matrix=newsales.iloc[:,24:36],
    grid_points=grid_points[24:36],
)
param_grid = {'n_neighbors': np.arange(1, 25),
              'weights': ['uniform', 'distance']}
knn = KNeighborsRegressor(metric='euclidean', multivariate_metric=True)
gscv = GridSearchCV(knn, param_grid, cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
gscv.fit(fd_x, fd_y)
print("Best params", gscv.best_params_)
print("Best score", gscv.best_score_)

Best params {'n_neighbors': 8, 'weights': 'distance'}
Best score 0.8840878702272456


In [15]:
knn = KNeighborsRegressor(n_neighbors=8, weights='distance')
train, test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_train = train.iloc[:,24:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[24:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
knn.fit(X_train, y_train)
X_test = test.iloc[:,24:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[24:36],
)
y_pred = knn.predict(X_test)
pred = pd.DataFrame(y_pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("FDA 12months mae:", mae/len(y_test))
print("FDA 12months rmse:", rmse/len(y_test)) 

FDA 12months mae: 69.69359268766571
FDA 12months rmse: 82.07114524426872


In [ ]:
#FDA 24months

In [16]:
fd_y = skfda.FDataGrid(
    data_matrix=newsales.iloc[:,36:40],
    grid_points=grid_points[36:40],
)
fd_x = skfda.FDataGrid(
    data_matrix=newsales.iloc[:,12:36],
    grid_points=grid_points[12:36],
)
param_grid = {'n_neighbors': np.arange(1, 25),
              'weights': ['uniform', 'distance']}
knn = KNeighborsRegressor(metric='euclidean', multivariate_metric=True)
gscv = GridSearchCV(knn, param_grid, cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
gscv.fit(fd_x, fd_y)
print("Best params", gscv.best_params_)
print("Best score", gscv.best_score_)

Best params {'n_neighbors': 12, 'weights': 'distance'}
Best score 0.8784006631528374


In [17]:
knn = KNeighborsRegressor(n_neighbors=12, weights='distance')
train, test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_train = train.iloc[:,12:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[12:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
knn.fit(X_train, y_train)
X_test = test.iloc[:,12:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[12:36],
)
y_pred = knn.predict(X_test)
pred = pd.DataFrame(y_pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("FDA 24months mae:", mae/len(y_test))
print("FDA 24months rmse:", rmse/len(y_test)) 

FDA 24months mae: 68.9481979765459
FDA 24months rmse: 81.10053108899012


In [18]:
#FDA 36months

In [19]:
fd_y = skfda.FDataGrid(
    data_matrix=newsales.iloc[:,36:40],
    grid_points=grid_points[36:40],
)
fd_x = skfda.FDataGrid(
    data_matrix=newsales.iloc[:,0:36],
    grid_points=grid_points[0:36],
)
param_grid = {'n_neighbors': np.arange(1, 25),
              'weights': ['uniform', 'distance']}
knn = KNeighborsRegressor(metric='euclidean', multivariate_metric=True)
gscv = GridSearchCV(knn, param_grid, cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
gscv.fit(fd_x, fd_y)
print("Best params", gscv.best_params_)
print("Best score", gscv.best_score_)

Best params {'n_neighbors': 7, 'weights': 'distance'}
Best score 0.8762306134660605


In [20]:
knn = KNeighborsRegressor(n_neighbors=7, weights='distance')
train, test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_train = train.iloc[:,0:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[0:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
knn.fit(X_train, y_train)
X_test = test.iloc[:,0:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[0:36],
)
y_pred = knn.predict(X_test)
pred = pd.DataFrame(y_pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("FDA 36months mae:", mae/len(y_test))
print("FDA 36months rmse:", rmse/len(y_test)) 

FDA 36months mae: 69.96096983717943
FDA 36months rmse: 81.11569609717593


In [21]:
#Table 3: FDA models using different basis functions

In [22]:
#FDA fourier

In [23]:
'''
def Smape(y_true,y_pred):
    a1 = 2.0 * np.abs(y_true[37]-y_pred[0])/ (np.abs(y_true[37])+np.abs(y_pred[0]))
    a2 = 2.0 * np.abs(y_true[38]-y_pred[1])/ (np.abs(y_true[38])+np.abs(y_pred[1]))
    a3 = 2.0 * np.abs(y_true[39]-y_pred[2])/ (np.abs(y_true[39])+np.abs(y_pred[2]))
    a4 = 2.0 * np.abs(y_true[40]-y_pred[3])/ (np.abs(y_true[40])+np.abs(y_pred[3]))
    return (a1+a2+a3+a4)/4
'''
knn = KNeighborsRegressor(n_neighbors=11, weights='distance')
train, test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_train = train.iloc[:,12:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[12:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
fourier_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.Fourier(  # Fourier
        n_basis=375,
        domain_range=X_train.domain_range,
    ),
])
X_train_fourier = X_train.to_basis(fourier_basis)
X_train_grid = X_train_fourier.to_grid(grid_points = grid_points[12:36])
knn.fit(X_train_grid, y_train)
X_test = test.iloc[:,12:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[12:36],
)
X_test_fourier = X_test.to_basis(fourier_basis)
X_test_grid = X_test_fourier.to_grid(grid_points = grid_points[12:36])
y_pred = knn.predict(X_test_grid)
pred = y_pred.to_grid(grid_points = grid_points[36:40])
pred = pd.DataFrame(pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
#smape = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
    #smape = smape + Smape(y_test.loc[i], pred.loc[i])
print("mae:", round(mae/len(y_test),2))
#print("smape:", round(smape/len(y_test),2))
print("rmse:", round(rmse/len(y_test),2))

mae: 68.54
rmse: 80.54


In [24]:
#FDA bspline

In [25]:
knn = KNeighborsRegressor(n_neighbors=12, weights='distance')
train, test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_train = train.iloc[:,12:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[12:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
bspline_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.BSpline(  # BSpline
        n_basis=350,
        domain_range=X_train.domain_range,
    ),
])
X_train_bspline = X_train.to_basis(bspline_basis)
X_train_grid = X_train_bspline.to_grid(grid_points = grid_points[12:36])
knn.fit(X_train_grid, y_train)
X_test = test.iloc[:,12:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[12:36],
)
X_test_bspline = X_test.to_basis(bspline_basis)
X_test_grid = X_test_bspline.to_grid(grid_points = grid_points[12:36])
y_pred = knn.predict(X_test_grid)
pred = y_pred.to_grid(grid_points = grid_points[36:40])
pred = pd.DataFrame(pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("mae:", mae/len(y_test))
print("rmse:", rmse/len(y_test)) 

mae: 68.94819797654591
rmse: 81.10053108899012


In [26]:
#Table 4: FDA models using different training variables

In [27]:
#FDA sales = FDA fourier
#FDA MSRP

In [28]:
data_matrix_train_MSRP = pd.read_csv('data_matrix_train_MSRP.csv')
del data_matrix_train_MSRP['Unnamed: 0']
data_matrix_test_MSRP = pd.read_csv('data_matrix_test_MSRP.csv')
del data_matrix_test_MSRP['Unnamed: 0']
MSRP = pd.concat([data_matrix_train_MSRP,data_matrix_test_MSRP], axis=1)
MSRP.columns = range(1,41)
newMSRP = MSRP[err_fd_y < err_thresh]

In [29]:
fd_y = skfda.FDataGrid(
    data_matrix=newsales,
    grid_points=grid_points,
)
fd_x = skfda.FDataGrid(
    data_matrix=newMSRP,
    grid_points=grid_points,
)
fourier_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.Fourier(  # Fourier
        n_basis=375,
        domain_range=fd_x.domain_range,
    ),
])
fd_x_fourier = fd_x.to_basis(fourier_basis)
fd_x_grid = fd_x_fourier.to_grid(grid_points = grid_points[12:36])
param_grid = {'n_neighbors': np.arange(1, 25),
              'weights': ['uniform', 'distance']}
knn = KNeighborsRegressor(metric='euclidean', multivariate_metric=True)
gscv = GridSearchCV(knn, param_grid, cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
gscv.fit(fd_x_grid, fd_y)  
print("Best params", gscv.best_params_)
print("Best score", gscv.best_score_)

Best params {'n_neighbors': 9, 'weights': 'uniform'}
Best score 0.22280278833673695


In [4]:
#FDA+线性 vs FDA+KNN

In [105]:
train, test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_test = test.iloc[:,24:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[24:36],
)
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
t = np.linspace(25, 40, num = 26)
mae = 0
rmse = 0
for i in range(len(X_test)):
    values = X_test[i](t, extrapolation="periodic")[..., 0]
    mae = mae + mean_absolute_error(y_test.loc[i], pd.DataFrame(values).loc[0][22:])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pd.DataFrame(values).loc[0][22:]))
print("extrapolation mae:", mae/len(y_test))
print("extrapolation rmse:", rmse/len(y_test)) 

extrapolation mae: 114.76732101616624
extrapolation rmse: 144.03323075919627


In [104]:
pd.DataFrame(values)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,70.0,31.0,12.6,35.4,41.0,38.0,43.4,45.8,42.2,34.6,...,70.4,72.6,104.4,44.0,5.0,27.8,42.0,39.0,41.6,47.0


In [30]:
knn = KNeighborsRegressor(n_neighbors=9, weights='distance')
y_train, y_test = train_test_split(newsales, test_size = 0.2, random_state=10)
X_train, X_test = train_test_split(newMSRP, test_size = 0.2, random_state=10)
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points,
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points,
)
fourier_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.Fourier(  # Fourier
        n_basis=375,
        domain_range=X_train.domain_range,
    ),
])
X_train_fourier = X_train.to_basis(fourier_basis)
X_train_grid = X_train_fourier.to_grid(grid_points = grid_points[12:36])
knn.fit(X_train_grid, y_train)
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points,
)
X_test_fourier = X_test.to_basis(fourier_basis)
X_test_grid = X_test_fourier.to_grid(grid_points = grid_points[12:36])
y_pred = knn.predict(X_test_grid)
pred = y_pred.to_grid(grid_points = grid_points)
pred = pd.DataFrame(pred.data_matrix.reshape((test.shape[0],40)))
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("MSRP mae:", mae/len(y_test))
print("MSRP rmse:", rmse/len(y_test)) 

MSRP mae: 155.97926695048216
MSRP rmse: 192.88158184340364


In [31]:
#Table 5: compare results with benchmark models
#FDA = FDA sales = FDA fourier

In [32]:
#SARIMA

In [49]:
import warnings
warnings.filterwarnings("ignore")
train, test = train_test_split(sales, test_size = 0.2, random_state=10)
mae = 0
rmse = 0
for i in test.index:
    m = SARIMAX(test.loc[i][0:36], order = (0,1,1), seasonal_order = (0,1,1,12)).fit(disp=0)
    data_predict = m.predict(37,40)
    mae = mae + mean_absolute_error(test.loc[i][36:40], data_predict)
    rmse = rmse + sqrt(mean_squared_error(test.loc[i][36:40], data_predict))
print("mae:", mae/len(test))
print("rmse:", rmse/len(test))

mae: 175.8517351878092
rmse: 196.11885140579219


In [33]:
#ANN
#初始值随机，每次运行结果会不一样

In [6]:
from numpy.random import seed 
seed(1)
train, test = train_test_split(sales, test_size = 0.2, random_state=10)
X_train = train.iloc[:,0:36]
y_train = train.iloc[:,36:40]
X_test = test.iloc[:,0:36]
y_test = test.iloc[:,36:40]
nn_model = Sequential()
nn_model.add(Dense(15,input_dim=X_train.shape[1],activation='relu'))
nn_model.add(Dense(4))
nn_model.compile(loss='mean_squared_error',optimizer='adam')
early_stop = EarlyStopping(monitor='loss',patience=2,verbose=1)
final_mae = 0
final_rmse = 0
for m in range(0,70):
    nn_model.fit(X_train, y_train, epochs=5, batch_size=len(X_train), verbose=0, callbacks=[early_stop], shuffle=False)
    pred = nn_model.predict(X_test)
    y_test = y_test.reset_index(drop = True)
    mae_ann = 0
    rmse_ann = 0
    for i in y_test.index:
        mae_ann = mae_ann + mean_absolute_error(y_test.iloc[i][[37,38,39,40]], pd.DataFrame(pred).iloc[i])
        rmse_ann = rmse_ann + sqrt(mean_squared_error(y_test.iloc[i][[37,38,39,40]], pd.DataFrame(pred).iloc[i]))
    final_mae = final_mae + mae_ann/len(y_test)
    final_rmse = final_rmse + rmse_ann/len(y_test)
print("mae_ann:", final_mae/70)
print("rmse_ann:", final_rmse/70)

mae_ann: 108.46326575791414
rmse_ann: 128.40311667787768


In [34]:
#LSTM
#初始值随机，每次运行结果会不一样

In [13]:
X_train_lstm = np.array(X_train).reshape(X_train.shape[0],1,X_train.shape[1])
lstm_model = Sequential()
lstm_model.add(LSTM(12,input_shape=(1,X_train_lstm.shape[2]),activation='relu',kernel_initializer='lecun_uniform',return_sequences=False))
lstm_model.add(Dense(4))
lstm_model.compile(loss='mean_squared_error',optimizer='adam')
early_stop = EarlyStopping(monitor='loss',patience=2,verbose=1)
final_mae = 0
final_rmse = 0
for m in range(0,50):
    lstm_model.fit(X_train_lstm, y_train, epochs=20, batch_size=len(X_train_lstm), verbose=0, callbacks=[early_stop], shuffle=False)
    X_test_lstm = np.array(X_test).reshape(X_test.shape[0],1,X_test.shape[1])
    pred = lstm_model.predict(X_test_lstm)
    y_test = y_test.reset_index(drop = True)
    mae_lstm = 0
    rmse_lstm = 0
    for i in y_test.index:
        mae_lstm = mae_lstm + mean_absolute_error(y_test.iloc[i][[37,38,39,40]], pd.DataFrame(pred).iloc[i])
        rmse_lstm = rmse_lstm + sqrt(mean_squared_error(y_test.iloc[i][[37,38,39,40]], pd.DataFrame(pred).iloc[i]))
    final_mae = final_mae + mae_lstm/len(y_test)
    final_rmse = final_rmse + rmse_lstm/len(y_test)
print("mae_lstm:", final_mae/50)
print("rmse_lstm:", final_rmse/50)

Epoch 00010: early stopping
Epoch 00005: early stopping
mae_lstm: 87.57952098186168
rmse_lstm: 101.36044335363593


In [35]:
#Table 6: MAE, SMAPE, and RMSE scores of models using data with different granularity
#FDA province = FDA = FDA sales = FDA fourier

In [37]:
#FDA city

In [38]:
data_matrix_city = pd.read_csv('data_matrix_city.csv')
del data_matrix_city['Unnamed: 0']
grid_points = []
for x in range(25,41):
    grid_points.append(x)
fd = skfda.FDataGrid(
    data_matrix=data_matrix_city,
    grid_points=grid_points,
)
q = 1
fpca_clean = FPCA(n_components=q)
fpca_clean.fit(fd)
fd_hat = fpca_clean.inverse_transform(
    fpca_clean.transform(fd)
)
err_fd = l2_distance(
    fd,
    fd_hat
) / l2_norm(fd)
x_density = np.linspace(0., 1.6, num=1972)
density_err_fd = gaussian_kde(err_fd)
err_thresh = np.quantile(err_fd, 0.95)
newsales_city = data_matrix_city[err_fd < err_thresh]
newsales_city.columns = range(25,41)
def Smape(y_true,y_pred):
    a1 = 2.0 * np.abs(y_true[37]-y_pred[0])/ (np.abs(y_true[37])+np.abs(y_pred[0]))
    a2 = 2.0 * np.abs(y_true[38]-y_pred[1])/ (np.abs(y_true[38])+np.abs(y_pred[1]))
    a3 = 2.0 * np.abs(y_true[39]-y_pred[2])/ (np.abs(y_true[39])+np.abs(y_pred[2]))
    a4 = 2.0 * np.abs(y_true[40]-y_pred[3])/ (np.abs(y_true[40])+np.abs(y_pred[3]))
    return (a1+a2+a3+a4)/4
knn = KNeighborsRegressor(n_neighbors=10, weights='distance')
train, test = train_test_split(newsales_city, test_size = 0.2, random_state=10)
X_train = train.iloc[:,0:12]
y_train = train.iloc[:,12:16]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[0:12],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[12:16],
)
bspline_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.BSpline(  # BSpline
        n_basis=350,
        domain_range=fd_x.domain_range,
    ),
])
X_train_bspline = X_train.to_basis(bspline_basis)
X_train_grid = X_train_bspline.to_grid(grid_points = grid_points[0:12])
knn.fit(X_train_grid, y_train)
X_test = test.iloc[:,0:12]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[0:12],
)
X_test_bspline = X_test.to_basis(bspline_basis)
X_test_grid = X_test_bspline.to_grid(grid_points = grid_points[0:12])
y_pred = knn.predict(X_test_grid)
pred = y_pred.to_grid(grid_points = grid_points[12:16])
pred = pd.DataFrame(pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,12:16]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
smape = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    smape = smape + Smape(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("mae:", round(mae/len(y_test),2))
print("smape:", round(smape/len(y_test),2))
print("rmse:", round(rmse/len(y_test),2))

mae: 12.78
smape: 0.35
rmse: 15.49


In [39]:
#FDA city hierarchy

In [40]:
data_matrix_citylevel = pd.read_csv('data_matrix_citylevel.csv')
del data_matrix_citylevel['Unnamed: 0']
grid_points = []
for x in range(1,41):
    grid_points.append(x)
fd = skfda.FDataGrid(
    data_matrix=data_matrix_citylevel,
    grid_points=grid_points,
)
q = 2
fpca_clean = FPCA(n_components=q)
fpca_clean.fit(fd)
fd_hat = fpca_clean.inverse_transform(
    fpca_clean.transform(fd)
)
err_fd = l2_distance(
    fd,
    fd_hat
) / l2_norm(fd)
x_density = np.linspace(0., 1.6, num=997)
density_err_fd = gaussian_kde(err_fd)
err_thresh = np.quantile(err_fd, 0.95)
newcitylevel = data_matrix_citylevel[err_fd < err_thresh]
newcitylevel.columns = range(1,41)
def Smape(y_true,y_pred):
    a1 = 2.0 * np.abs(y_true[37]-y_pred[0])/ (np.abs(y_true[37])+np.abs(y_pred[0]))
    a2 = 2.0 * np.abs(y_true[38]-y_pred[1])/ (np.abs(y_true[38])+np.abs(y_pred[1]))
    a3 = 2.0 * np.abs(y_true[39]-y_pred[2])/ (np.abs(y_true[39])+np.abs(y_pred[2]))
    a4 = 2.0 * np.abs(y_true[40]-y_pred[3])/ (np.abs(y_true[40])+np.abs(y_pred[3]))
    return (a1+a2+a3+a4)/4
knn = KNeighborsRegressor(n_neighbors=5, weights='distance')
train, test = train_test_split(newcitylevel, test_size = 0.2, random_state=10)
X_train = train.iloc[:,0:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[0:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
fourier_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.Fourier(  # Fourier
        n_basis=475,
        domain_range=X_train.domain_range,
    ),
])
X_train_fourier = X_train.to_basis(fourier_basis)
X_train_grid = X_train_fourier.to_grid(grid_points = grid_points[0:36])
knn.fit(X_train_grid, y_train)
X_test = test.iloc[:,0:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[0:36],
)
X_test_fourier = X_test.to_basis(fourier_basis)
X_test_grid = X_test_fourier.to_grid(grid_points = grid_points[0:36])
y_pred = knn.predict(X_test_grid)
pred = y_pred.to_grid(grid_points = grid_points[36:40])
pred = pd.DataFrame(pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
smape = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    smape = smape + Smape(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("mae:", round(mae/len(y_test),2))
print("smape:", round(smape/len(y_test),2))
print("rmse:", round(rmse/len(y_test),2))

mae: 180.15
smape: 0.28
rmse: 202.54


In [41]:
#FDA type submarket

In [42]:
data_matrix_submarket = pd.read_csv('data_matrix_submarket.csv')
del data_matrix_submarket['Unnamed: 0']
data_matrix_submarket.columns = range(1,41)
fd = skfda.FDataGrid(
    data_matrix=data_matrix_submarket,
    grid_points=grid_points,
)
q = 1
fpca_clean = FPCA(n_components=q)
fpca_clean.fit(fd)
fd_hat = fpca_clean.inverse_transform(
    fpca_clean.transform(fd)
)
err_fd = l2_distance(
    fd,
    fd_hat
) / l2_norm(fd)
x_density = np.linspace(0., 1.6, num=257)
density_err_fd = gaussian_kde(err_fd)
err_thresh = np.quantile(err_fd, 0.99)
newsubmarket = data_matrix_submarket[err_fd < err_thresh]
def Smape(y_true,y_pred):
    a1 = 2.0 * np.abs(y_true[37]-y_pred[0])/ (np.abs(y_true[37])+np.abs(y_pred[0]))
    a2 = 2.0 * np.abs(y_true[38]-y_pred[1])/ (np.abs(y_true[38])+np.abs(y_pred[1]))
    a3 = 2.0 * np.abs(y_true[39]-y_pred[2])/ (np.abs(y_true[39])+np.abs(y_pred[2]))
    a4 = 2.0 * np.abs(y_true[40]-y_pred[3])/ (np.abs(y_true[40])+np.abs(y_pred[3]))
    return (a1+a2+a3+a4)/4
knn = KNeighborsRegressor(n_neighbors=3, weights='distance')
train, test = train_test_split(newsubmarket, test_size = 0.2, random_state=10)
X_train = train.iloc[:,23:36]
y_train = train.iloc[:,36:40]
X_train = skfda.FDataGrid(
    data_matrix=X_train,
    grid_points=grid_points[23:36],
)
y_train = skfda.FDataGrid(
    data_matrix=y_train,
    grid_points=grid_points[36:40],
)
bspline_basis = skfda.representation.basis.VectorValued([
    skfda.representation.basis.BSpline(  # BSpline
        n_basis=350,
        domain_range=X_train.domain_range,
    ),
])
X_train_bspline = X_train.to_basis(bspline_basis)
X_train_grid = X_train_bspline.to_grid(grid_points = grid_points[23:36])
knn.fit(X_train_grid, y_train)
X_test = test.iloc[:,23:36]
X_test = skfda.FDataGrid(
    data_matrix=X_test,
    grid_points=grid_points[23:36],
)
X_test_bspline = X_test.to_basis(bspline_basis)
X_test_grid = X_test_bspline.to_grid(grid_points = grid_points[23:36])
y_pred = knn.predict(X_test_grid)
pred = y_pred.to_grid(grid_points = grid_points[36:40])
pred = pd.DataFrame(pred.data_matrix.reshape((test.shape[0],4)))
y_test = test.iloc[:,36:40]
y_test = y_test.reset_index(drop = True)
mae = 0
rmse = 0
smape = 0
for i in y_test.index:
    mae = mae + mean_absolute_error(y_test.loc[i], pred.loc[i])
    smape = smape + Smape(y_test.loc[i], pred.loc[i])
    rmse = rmse + sqrt(mean_squared_error(y_test.loc[i], pred.loc[i]))
print("mae:", round(mae/len(y_test),2))
print("smape:", round(smape/len(y_test),2))
print("rmse:", round(rmse/len(y_test),2))

mae: 736.8
smape: 0.27
rmse: 882.24
